In [252]:
import functools
import itertools
import logging
import math
import os
import pickle
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import seaborn as sns
import yaml

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 12.0)})
sns.set_style("whitegrid")

import numpy as np
import pandas as pd
import torch.nn.functional as F

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [541]:
from lda4rec.datasets import Interactions, DataLoader, random_train_test_split
from lda4rec.evaluations import mrr_score, precision_recall_score, auc_score, summary
from lda4rec.estimators import MFEst, PopEst, LDA4RecEst, SNMFEst
from lda4rec.utils import process_ids, cmp_ranks

In [254]:
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
from pyro.distributions import constraints
from pyro.infer import SVI, Predictive, Trace_ELBO, TraceEnum_ELBO, config_enumerate

In [255]:
import neptune
# init dummy neptune to avoid problems with logging
neptune.init('a/b', backend=neptune.OfflineBackend()) 

In [360]:
from icecream import ic, install

install()
# configure icecream
def ic_str(obj):
    if hasattr(obj, "shape"):
        return f"{obj} "  #
    else:
        return str(obj)

In [361]:
ic.configureOutput(argToStringFunction=ic_str)

In [258]:
loader = DataLoader()
data = loader.load_movielens("100k")

In [259]:
max_interactions = 200
data.max_user_interactions_(max_interactions)

In [260]:
data.implicit_(0.)
train, test = random_train_test_split(data)

In [261]:
pop_est = PopEst()
pop_est.fit(train)

In [262]:
summary(pop_est, train=train, test=test)

,train,test
metric,,
prec,0.290000,0.104762
recall,0.053711,0.073714
mrr,0.026169,0.036003
auc,0.912129,0.875571


In [263]:
lda_est = LDA4RecEst(embedding_dim=4, n_iter=1000, batch_size=128, learning_rate=0.01, use_jit=True, alpha=1.)

In [42]:
lda_est.fit(train)

INFO:lda4rec.estimators:Epoch     0: loss 522180.90625


KeyboardInterrupt: 

In [264]:
summary(lda_est, train=train, test=test)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [669]:
mf_est = MFEst(embedding_dim=8, n_iter=30)
mf_est.fit(train)

INFO:lda4rec.estimators:Epoch     0: loss 0.3799139838766407
INFO:lda4rec.estimators:Epoch     1: loss 0.1577877906931413
INFO:lda4rec.estimators:Epoch     2: loss 0.11146449069114955
INFO:lda4rec.estimators:Epoch     3: loss 0.09271160675263082
INFO:lda4rec.estimators:Epoch     4: loss 0.08099528727297847
INFO:lda4rec.estimators:Epoch     5: loss 0.07438334300610665
INFO:lda4rec.estimators:Epoch     6: loss 0.06834076501026347
INFO:lda4rec.estimators:Epoch     7: loss 0.06352411905756673
INFO:lda4rec.estimators:Epoch     8: loss 0.06052093864292712
INFO:lda4rec.estimators:Epoch     9: loss 0.06094917182785434
INFO:lda4rec.estimators:Epoch    10: loss 0.05869474932047966
INFO:lda4rec.estimators:Epoch    11: loss 0.05638193707107692
INFO:lda4rec.estimators:Epoch    12: loss 0.05733540744294186
INFO:lda4rec.estimators:Epoch    13: loss 0.0549502330479791
INFO:lda4rec.estimators:Epoch    14: loss 0.053756237563651964
INFO:lda4rec.estimators:Epoch    15: loss 0.05417848394529239
INFO:lda4r

0.04734043982257513

In [705]:
summary(mf_est, train=train, test=test)

,train,test
metric,,
prec,0.332951,0.117241
recall,0.061688,0.087535
mrr,0.029566,0.040600
auc,0.964943,0.898138


In [706]:
user_id = 128
orig_scores = mf_est.predict(np.array([user_id], dtype=np.int))
item_probs = mf_est.get_item_probs(user_id)

In [707]:
cmp_ranks(orig_scores, item_probs, eps=1e-5)

True

In [708]:
torch.where

<function _VariableFunctionsClass.where>

In [709]:
np.sum(np.argsort(orig_scores) != np.argsort(item_probs))

8

In [710]:
torch.min(torch.eye(10), dim=0)

torch.return_types.min(
values=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
indices=tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [711]:
torch.zeros(1).shape

torch.Size([1])

In [712]:
torch.min(torch.arange(10)-1.).shape

torch.Size([])

In [713]:
torch.zeros(0)

tensor([])

In [60]:
n_samples = 200
rng = np.random.default_rng(42)
model_params["interactions"] = data.interactions

predictive = Predictive(model, guide=guide, num_samples=n_samples)
samples = {
    k: v.detach().cpu().numpy()
    for k, v in predictive(**model_params).items()
}

IndexError: index 1 is out of bounds for dimension 0 with size 1